In [1]:
import pandas as pd

model_key = 'llama8b'

df = pd.read_pickle(model_key+'-zero-shot.pkl')

df


,doi,texto,name,collectionType,collectionSite,collectionSpecie,bioActivity,bioActivity_llm,collectionSite_llm,collectionSpecie_llm,collectionType_llm,name_llm
0,10.1016/S0031-9422(00)00248-X,10.1016@S0031-9422(00)00248-X.pdf\nPage 1\nPyr...,"[2'-hydroxy-3,4,4'-trimethoxy-5',6'-(2'',2''-d...",[Plant Isolated],[N/A/ES],[Neoraputia magnifica (Rutaceae)],[Antitrypanosomal],"[6, 7]","[6, 7]","[1]\n2'-Hydroxy-3,4,4',5-tetramethylpyrano)cha...","[6, 7]","[3',4'-Methylenedioxy-5,6,7-trimethoxyflavone ..."
1,10.1002/ptr.1249,10.1002@ptr.1249.pdf\nPage 1\nAntioxidant Flav...,"[Ourateacatechin, (-)-Epigallocatechin, Proant...",[Plant Isolated],[Ribeirao Preto/SP],[Maytenus aquifolium (Celastraceae)],[Antioxidant],"['antioxidant', 'antioxidant', 'antioxidant', ...","[Ribeirao Preto, Sao Paulo]",[Maytenus aquifolium Martius (Celastraceae)],"['Ethnopharmacological', 'traditional','medici...","[epigallocatechin(1), (+) ouratea-catechin (2)..."
2,10.1016/0031-9422(73)85048-4,10.1016@0031-9422(73)85048-4.pdf\nPage 1\nIso-...,"[7-Hydroxy-4',6-dimethoxyisoflavone; Afrormosi...",[Plant Isolated],[N/A/AM],[Dalbergia riparia],[Antitrypanosomal],"['Antifungal', 'antimicrobial', 'antimicrobial...","['coracao de negro' or 'pau-ferro', Linhares R...","[[""Dalbergia riparia"" (Dalbergiaceae), ""Poecil...","[coracao de negro, pau-ferro, Poecilanthe parv...","['tectorigenin', 'dalbergione', '(S)-4-methoxy..."
3,10.1021/np0303963,10.1021@np0303963.pdf\nPage 1\nFurther Bioacti...,[(-)-3-O-acetyl-spectaline],[Plant Isolated],[Sao Paulo/SP],[Cassia spectabilis (Fabaceae)],[Cytotoxic],"[antimicrobial, antitumor, DNA-damaging]",[Brazil],[Cassia spectabilis],"[Fruits, Flowers]","[(-)-3-O-Acetyl spectatine (1), (-)-7-hydroxyp..."
4,10.4067/S0717-97072009000300004,10.4067@S0717-97072009000300004.pdf\nPage 1\nI...,[Taraxerol acetate],[Plant Isolated],[Sao Paulo/SP],[Pterogyne nitens (Fabaceae)],[Inhibition of Glycosidase],"[antimicrobial, antioxidant, anti-inflammatory...",[Pterogyne niens],[Pterogyne nitens (Fabaceae-Caesalpinioideae)],[Collection type: plant],"[β-amyrin, lupenone, taraxerol, α-amyrin aceta..."
...,...,...,...,...,...,...,...,...,...,...,...,...
138,10.1080/14786410802243271,10.1080@14786410802243271.pdf\nPage 1\nTo cite...,"[4-Hydroxy-(3',7'-dimethyl-1'-oxo-octa-2'-E-6'...",[Plant Isolated],[N/A/SP],[Piper crassinervium (Piperaceae)],[Antitrypanosomal],"['antichagasic', 'antifungal', 'antileishmanic...","[Vale do Ribeira, Atlantic Forest (Sao Paulo S...",[Piper crassinervium (Piperaceae)],"['Wild', 'Cultivated', 'Wild', 'Cultivated', '...","['1,4-dihydroxy-2-(3',7'-dimethyl-1'-oxo-2'-E-..."
139,10.1590/S0100-40422006000300016,10.1590@S0100-40422006000300016.pdf\nPage 1\nC...,[(-)-Epigallocatechin-3-O-gallate],[Plant Isolated],[Ibate/SP],[Chrysophyllum marginatum (Sapotaceae)],[Antioxidant],"[antioxidant, antioxidant, antioxidant]","[Fortaleza-Ripasa Farm, in Ibate - SP, Brazil]","['Chrysophyllum marginatum', Sapotaceae, None]",[Sapotaceae],"[(lpha)-amirin, gallic acid, myricirin, querc..."
140,10.1016/j.cbi.2010.08.008,10.1016@j.cbi.2010.08.008.pdf\nPage 1\nCaseari...,"[Casearin X; (1R*,3S*,5S*,6aR*,7S*,8S*,10R*,10...",[Plant Isolated],[Sao Miguel Arcanjo/SP],[Casearia sylvestris (Flacourtiaceae)],"[Cytotoxic, Anticancer]","[22] A.G. Santos, P.M.P. Ferreira, G.M. Vieira...","[Sao Miguel Arcanjo, Sao Paulo, Brazil]","[(Casearin X, 0.7 μM), (Casearin X, 1.5 μM), (...","[(Casearin X, 0.7, 1.5, 3.0), (Doxorubicin, 0....","[Casearin X, 0.7, 1.5, 3.0]"
141,10.1016/S0031-9422(01)00431-9,10.1016@S0031-9422(01)00431-9.pdf\nPage 1\nAnt...,"[Fagaramide, Methyl (E)-6,7,8-trimethoxycinnam...",[Plant Isolated],"[Araraquara/SP, Manaus/AM]","[Piper tuberculatum (Piperaceae), Piper arbore...",[Antifungal],[1]\n[2] Antifungal amides from Piper arboreum...,"['Piper tuberculatum Jacq. seeds, leaves and s...","[1] N-[10-(13,14-Methylenedioxyphenyl)-7(E),9(...","[Pellitorine, dihydro piperine, dihydropiplart...","[N-[10-(13,14-methylenedioxyphenyl)-7(E),9(Z)-..."


In [2]:
estagios = ['1st', '2nd', '3rd', '4th']
folds = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
tarefas = ['bioActivity', 'collectionSite', 'collectionSpecie', 'collectionType', 'name']

hit_k = {
    "name": [],
    "bioActivity": [],
    "collectionSpecie": [],
    "collectionType": [],
    "collectionSite": []
}

k = {
    "name": 50,
    "bioActivity": 5,
    "collectionSpecie": 50,
    "collectionType": 1,
    "collectionSite": 20
}

import numpy as np
def hits_at(k, true, list_pred):
    hits = []
    for t in true:
        hit = False
        for index_lp, lp in enumerate(list_pred):
            if index_lp >= k:
                break
            if t == lp:
                hits.append(1)
                hit = True
                break
        if not(hit):
            hits.append(0)
    return np.mean(hits)

for tarefa in tarefas:
    for estagio in estagios:
        l_folds = []
        for fold in folds:
            name_file = 'splits/test_doi_' + tarefa + '_' + fold + '_' + estagio + '.csv'
            data = pd.read_csv(name_file) 
            list_doi = data['node'].to_list()
            df_filtrado = df[df['doi'].isin(list_doi)]
            l = []
            preds_list = []
            for index, row in df_filtrado.iterrows():
                pred_j = row[tarefa + '_llm']
                try:
                    pred = eval(str(pred_j))
                except:
                    pred = ['']
                true_j = row[tarefa]
                resultado_fold = hits_at(k[tarefa], true_j, pred)
                l.append(resultado_fold)
                preds_list.append(pred)
            with open(f"outputs_PT_ballena/{tarefa}_{estagio}_{fold}", 'a', encoding='utf-8') as f:
                f.write(str(model_key) + ': ' + str(preds_list) + '\n')

            l_folds.append(np.mean(l))
        print(tarefa, estagio, np.mean(l_folds))

<string>:1: SyntaxWarning: invalid escape sequence '\d'
<string>:1: SyntaxWarning: invalid escape sequence '\d'
<string>:1: SyntaxWarning: invalid escape sequence '\d'
<string>:1: SyntaxWarning: invalid escape sequence '\d'
<string>:1: SyntaxWarning: invalid escape sequence '\d'
<string>:1: SyntaxWarning: invalid escape sequence '\d'
<string>:1: SyntaxWarning: invalid escape sequence '\d'
<string>:1: SyntaxWarning: invalid escape sequence '\d'
<string>:1: SyntaxWarning: invalid escape sequence '\d'


bioActivity 1st 0.0
bioActivity 2nd 0.0
bioActivity 3rd 0.0
bioActivity 4th 0.0


collectionSite 1st 0.0069949270823368
collectionSite 2nd 0.007453703703703703
collectionSite 3rd 0.009191919191919192
collectionSite 4th 0.014814814814814814
collectionSpecie 1st 0.09646762811096568
collectionSpecie 2nd 0.09463366111238086


<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'str' object is not callable; perhaps 

collectionSpecie 3rd 0.0941029341029341
collectionSpecie 4th 0.08756613756613756
collectionType 1st 0.0
collectionType 2nd 0.0
collectionType 3rd 0.0
collectionType 4th 0.0
name 1st 0.04965955938553974
name 2nd 0.05340407838322505
name 3rd 0.059084737118846056
name 4th 0.07474747474747476
